# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846853577944                   -0.70    4.8
  2   -7.852321314158       -2.26       -1.53    1.0
  3   -7.852646234714       -3.49       -2.52    3.0
  4   -7.852646679527       -6.35       -3.36    2.8
  5   -7.852646686126       -8.18       -4.78    1.2
  6   -7.852646686724       -9.22       -5.28    3.2
  7   -7.852646686730      -11.25       -6.22    1.8
  8   -7.852646686730      -12.86       -7.63    2.0
  9   -7.852646686730      -14.75       -7.69    2.2
 10   -7.852646686730   +    -Inf       -8.78    2.2
 11   -7.852646686730   +    -Inf       -9.55    2.0
 12   -7.852646686730   +    -Inf      -10.12    2.2
 13   -7.852646686730   +  -14.75      -11.37    2.8
 14   -7.852646686730      -14.45      -11.73    1.8
 15   -7.852646686730   +  -14.75      -13.06    2.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846776499544                   -0.70           4.2
  2   -7.852539130662       -2.24       -1.63   0.80    2.2
  3   -7.852638208157       -4.00       -2.72   0.80    1.0
  4   -7.852646481275       -5.08       -3.35   0.80    1.8
  5   -7.852646678415       -6.71       -4.22   0.80    2.0
  6   -7.852646686566       -8.09       -4.75   0.80    1.8
  7   -7.852646686716       -9.82       -5.63   0.80    1.2
  8   -7.852646686730      -10.86       -6.15   0.80    2.2
  9   -7.852646686730      -12.62       -7.39   0.80    1.5
 10   -7.852646686730      -13.94       -7.97   0.80    2.2
 11   -7.852646686730   +  -15.05       -8.83   0.80    1.5
 12   -7.852646686730      -14.75      -10.01   0.80    2.0
 13   -7.852646686730   +    -Inf      -10.47   0.80    2.8
 14   -7.852646686730   +    -Inf      -11.91   0.80    1.2
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.442767e+01     3.909391e+00
 * time: 0.5550429821014404
     1     1.370069e+00     1.937601e+00
 * time: 0.8223450183868408
     2    -1.686579e+00     2.330617e+00
 * time: 0.8512780666351318
     3    -3.797308e+00     2.001718e+00
 * time: 0.8938901424407959
     4    -5.164471e+00     1.487741e+00
 * time: 0.939141035079956
     5    -6.883785e+00     9.920049e-01
 * time: 0.9787909984588623
     6    -6.985145e+00     1.331095e+00
 * time: 1.0088350772857666
     7    -7.584200e+00     1.078871e+00
 * time: 1.0375950336456299
     8    -7.661822e+00     1.130320e+00
 * time: 1.0664341449737549
     9    -7.758679e+00     1.073831e+00
 * time: 1.0951359272003174
    10    -7.776428e+00     1.124570e+00
 * time: 1.1234509944915771
    11    -7.795573e+00     1.028508e+00
 * time: 1.1518311500549316
    12    -7.820556e+00     2.174528e-01
 * time: 1.193424940109253
    13    -7.836095e+00     2.156783e-01
 * time: 1.222589015960

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846867605831                   -0.70    4.2
  2   -7.852307255586       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686559                   -2.55
  2   -7.852646686730       -9.77       -5.89
  3   -7.852646686730   +    -Inf      -11.62
  4   -7.852646686730   +    -Inf      -16.01


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.2945610626287292e-14
|ρ_newton - ρ_scfv| = 2.3517521623550683e-13
|ρ_newton - ρ_dm|   = 3.975715854132333e-9
